In [9]:
# Carica il file csv e converto colonna dei prezzi in int
import pandas as pd
import re

df=pd.read_csv("dati_palermo/palermo_dati_booking.csv")
df['prezzo'] = df['prezzo'].apply(lambda x: int(re.search(r'\d+', x).group()))
df.columns

Index(['nome_hotel', 'prezzo', 'stanza', 'città', 'data', 'punteggio',
       'numero_recensioni', 'distanza_centro', 'genius', 'offerte',
       'colazione_inclusa', 'senza_pagamento_anticipato',
       'cancellazione_gratuita', 'os', 'username', 'user_agent_type',
       'TIMESTAMP'],
      dtype='object')

In [10]:
# Crea delle colonne con valori True/False in base alle offerte
df['booking_paga'] = df['offerte'].str.contains('Booking.com paga')
df['offerta_tempo_limitato'] = df['offerte'].str.contains('tempo limitato')
df['prezzo_mobile'] = df['offerte'].str.contains('mobile')
df['offerta_super_segreta'] = df['offerte'].str.contains('SuperSegreta')
df['offerta_inizio_2024'] = df['offerte'].str.contains('Offerta Inizio 2024')
# Inserisce False nei campi nulli
df[['booking_paga','offerta_tempo_limitato','prezzo_mobile','offerta_super_segreta','offerta_inizio_2024']] = df[['booking_paga','offerta_tempo_limitato','prezzo_mobile','offerta_super_segreta','offerta_inizio_2024']].fillna(False)

In [11]:
# Visualizza lista ricerche in base ad orari
df['TIMESTAMP'].unique()

array(['03-03-2024---16:59', '03-03-2024---17:10', '03-03-2024---17:20',
       '03-03-2024---17:44', '03-03-2024---17:59', '03-03-2024---18:30',
       '03-04-2024---12:20', '03-04-2024---12:31', '03-04-2024---12:49'],
      dtype=object)

In [12]:
# Divide il dataframe in base a orari ricerche per fare poi il merge
orario1 = df["TIMESTAMP"].unique()[1]
orario2 = df["TIMESTAMP"].unique()[3]

df1 = df[df["TIMESTAMP"]==orario1]
df2 = df[df["TIMESTAMP"]==orario2]

# Eliminazione duplicati
df1 = df1.drop_duplicates()
df2 = df2.drop_duplicates()

In [13]:
# MERGE TRA DATAFRAME
'''
Se le ricerche sono dello stesso tipo (Mobile o Desktop) il merge avviene anche sulla colonna stanza
Vengono inoltre compattate eventuali colonne uguali e prese in considerazione le righe in cui vi è una differenza di prezzo
'''
df_list = df.columns.tolist()
df_list =  df_list[3:4] +df_list[5:-2]
if df1['user_agent_type'].unique()[0]==df2['user_agent_type'].unique()[0]:
    merged_df = pd.merge(df1, df2, on=['nome_hotel','stanza','data'], how='inner')
else:
    merged_df = pd.merge(df1, df2, on=['nome_hotel','data'], how='inner')
merged_df = merged_df[merged_df["prezzo_x"]!=merged_df["prezzo_y"]]
for col in df_list:
    if merged_df[col+"_x"].equals(merged_df[col+"_y"]):
        merged_df[col] = merged_df[col+"_x"]
        merged_df.drop(columns=[col+"_x",col+"_y"],inplace=True)
# Elimino hotel con stesso nome
merged_df = merged_df.drop_duplicates(subset="nome_hotel",keep= False)        
merged_df.columns

Index(['nome_hotel', 'prezzo_x', 'stanza_x', 'città_x', 'data', 'punteggio_x',
       'numero_recensioni_x', 'distanza_centro_x', 'genius_x', 'offerte_x',
       'colazione_inclusa_x', 'senza_pagamento_anticipato_x',
       'cancellazione_gratuita_x', 'os_x', 'username_x', 'user_agent_type_x',
       'TIMESTAMP_x', 'booking_paga_x', 'offerta_tempo_limitato_x',
       'prezzo_mobile_x', 'offerta_super_segreta_x', 'offerta_inizio_2024_x',
       'prezzo_y', 'stanza_y', 'città_y', 'punteggio_y', 'numero_recensioni_y',
       'distanza_centro_y', 'genius_y', 'offerte_y', 'colazione_inclusa_y',
       'senza_pagamento_anticipato_y', 'cancellazione_gratuita_y', 'os_y',
       'username_y', 'user_agent_type_y', 'TIMESTAMP_y', 'booking_paga_y',
       'offerta_tempo_limitato_y', 'prezzo_mobile_y',
       'offerta_super_segreta_y', 'offerta_inizio_2024_y'],
      dtype='object')

In [8]:
# Per visualizzare tutte le righe con differenze di prezzo
#pd.set_option('display.max_rows', None)
merged_df[["nome_hotel","prezzo_x","prezzo_y","offerte_x","offerte_y"]]
merged_df

,nome_hotel,prezzo_x,stanza_x,città_x,data,punteggio_x,numero_recensioni_x,distanza_centro_x,genius_x,offerte_x,...,cancellazione_gratuita_y,os_y,username_y,user_agent_type_y,TIMESTAMP_y,booking_paga_y,offerta_tempo_limitato_y,prezzo_mobile_y,offerta_super_segreta_y,offerta_inizio_2024_y
5,Eurostars Centrale Palace Hotel,108,Camera Matrimoniale Standard,"Monte di Pietà, Palermo",2024-03-20,"8,3",3.260 recensioni,"0,7 km dal centro",False,Offerta a tempo limitato,...,True,Android,pantilaura56,mobile,03-03-2024---17:44,False,False,True,False,False
15,Palermo Politeama rooms,68,Camera Matrimoniale Economy,"Ruggero Settimo, Palermo",2024-03-20,10,5 recensioni,"0,5 km dal centro",True,NaN,...,True,Android,pantilaura56,mobile,03-03-2024---17:44,False,False,True,False,False
17,Afea Art & Rooms,64,Camera Classic Matrimoniale/Doppia con Letti S...,"Ruggero Settimo, Palermo",2024-03-20,"8,9",562 recensioni,"0,6 km dal centro",False,Offerta a tempo limitato,...,True,Android,pantilaura56,mobile,03-03-2024---17:44,True,True,False,False,False
19,Alba e i Mori,50,Camera Matrimoniale con Bagno in Comune,Palermo,2024-03-20,"8,6",59 recensioni,"0,6 km dal centro",True,NaN,...,True,Android,pantilaura56,mobile,03-03-2024---17:44,False,False,True,False,False
27,Marmur Suite,80,Camera Matrimoniale,"Albergaria, Palermo",2024-03-20,"8,7",639 recensioni,1 km dal centro,True,NaN,...,True,Android,pantilaura56,mobile,03-03-2024---17:44,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
929,Casa Reale,245,Appartamento Superior,Palermo,2024-03-20,"9,2",6 recensioni,"1,3 km dal centro",False,NaN,...,True,Android,pantilaura56,mobile,03-03-2024---17:44,False,False,True,False,False
932,Zisas,104,Appartamento con 1 Camera da Letto,Palermo,2024-03-20,"9,2",36 recensioni,"1,4 km dal centro",False,NaN,...,True,Android,pantilaura56,mobile,03-03-2024---17:44,False,False,True,False,False
938,I GIRASOLI SHORT LETS,111,Appartamento con 3 Camere da Letto,Palermo,2024-03-20,"9,8",4 recensioni,"2,3 km dal centro",True,NaN,...,True,Android,pantilaura56,mobile,03-03-2024---17:44,False,False,True,False,False
942,Viola's Luxury House,125,Appartamento con 2 Camere da Letto,"Monte di Pietà, Palermo",2024-03-20,"8,7",3 recensioni,500 m dal centro,False,NaN,...,True,Android,pantilaura56,mobile,03-03-2024---17:44,False,False,True,False,False


In [ ]:
#aggiungo colonna con differenza prezzi
pd.set_option('display.max_rows', None)
merged_df["differenza_prezzo"] = merged_df["prezzo_x"]-merged_df["prezzo_y"]
merged_df.sort_values(by='differenza_prezzo',ascending= False)
merged_df[['nome_hotel','prezzo_x','prezzo_y','differenza_prezzo','offerte_x','offerte_y']]



In [ ]:
# Visualizza dove compare una specifica offerta in almeno una delle ricerche
df_mobile = merged_df[(merged_df['prezzo_mobile_x'])|(merged_df['prezzo_mobile_y'])]
len(df_mobile)
len(merged_df)


In [ ]:
# Elimino righe in cui il mobile non ha la stanza
if merged_df['user_agent_type_x'].unique()[0] == 'mobile':
    df_senza_stanza = merged_df['stanza_x'].dropna()
elif merged_df['user_agent_type_y'].unique()[0] == 'mobile':
    df_senza_stanza = merged_df['stanza_y'].dropna()
len(df_senza_stanza)